In [88]:
#import module
from collections import Counter
import numpy as np

In [89]:
# Build Node
class Node:
	def __init__(self, feature=None, value=None, results=None, true_branch=None, false_branch=None):
		self.feature = feature #Thuộc tính để phân chia
		self.value = value	 # Giá trị của thuộc tính phân chia
		self.results = results # Lưu label của class nếu nó là leave
		self.true_branch = true_branch # Rẽ sanh nhánh T-branch nếu giá trị đúng với thuộc tính
		self.false_branch = false_branch # Rẽ sang nhánh F-branch nếu giá trị sai với thuộc tính

In [90]:
'''Build function calculate gini
Tính số lần xuất hiện của data bằng cách sử dụng np.bincount
Tính xác suất của từng phần tử và dùng công thức đã trình bày để tính gini
'''

def gini(data):
	dem = np.bincount(data)
	probabilities = dem / len(data)
	gini_inpuri = 1 - np.sum([p*p for p in probabilities if p > 0])     
	return gini_inpuri

In [91]:
'''
Build split_data function
Dựa vào feature và value để split data
Giá trị <= thì qua nhánh true, giá trị > thì qua nhánh false
Trả về các subset gồm feature(true_X, false_X) và label(true_y, false_y)
'''

def split_data(X, y, feature, value):
	true_line = np.where(X[:, feature] <= value)[0]
	false_line = np.where(X[:, feature] > value)[0]
	true_X, true_y = X[true_line], y[true_line]
	false_X, false_y = X[false_line], y[false_line]
	return true_X, true_y, false_X, false_y

In [92]:
'''
Build tree function by CART algorithm
input là feature X và label y
Đầu tiên, kiểm tra các label có đồng nhất không, nếu có trả về leave có label tương ứng
Lặp qua các feature, tính gini, tính total_gini và chọn total_gini nhỏ nhất làm tiêu chí split data
Đệ quy để xây dựng các true_branch và false_branch
output là leave được gắn label tương ứng
'''

def decision_tree(X, y, math_depth = 0):
	if len(set(y)) == 1:
		return Node(results = y[0])

	if math_depth == 0:
		return Node(results = np.mean(y))
	
	best_total_gini = float('inf')
	tieu_chuan = None
	best_sets = None
	n_features = X.shape[1]

	#current_gini = gini(y)

	for feature in range(n_features):
		feature_values = set(X[:, feature])
		for value in feature_values:
			true_X, true_y, false_X, false_y = split_data(X, y, feature, value)
			true_gini = gini(true_y)
			false_gini = gini(false_y)
			p = len(true_y) / len(y)
			total_gini =  p * true_gini + (1 - p) * false_gini

			if total_gini < best_total_gini:
				best_total_gini = total_gini
				tieu_chuan = (feature, value)
				best_sets = (true_X, true_y, false_X, false_y)

	if best_sets is not None and best_total_gini < total_gini:
		T_branch = decision_tree(best_sets[0], best_sets[1], math_depth - 1)
		F_branch = decision_tree(best_sets[2], best_sets[3], math_depth - 1)
		return Node(feature = tieu_chuan[0], value = tieu_chuan[1], true_branch = T_branch, false_branch = F_branch)

	return Node(results = np.mean(y))


In [93]:
'''
Build predict function
Đệ quy cho đến khi tìm thấy leave, trả về label ứng với nó
'''
def predict(tree, sample):
	if tree.results is not None:
		return tree.results
	else:
		branch = tree.false_branch
		if sample[tree.feature] <= tree.value:
			branch = tree.true_branch
		return predict(branch, sample)

In [94]:
#Testing
X = np.array([[2, 2], [2, 1], [2, 0], [1, 1],[1, 2], [1, 0], [0, 0], [0, 1], [0, 2], [6, 6]])
y = np.array([11, 32, 93, 4, 65, 26, 87, 48, 19, 70])

# Building the tree
tree = decision_tree(X, y, math_depth = 5)  #có thể thay đổi math_depth tùy vào kích thước dữ liệu

for sample in X:
    prediction = predict(tree, sample)
    print(f"Prediction for sample {sample}: {prediction}")

Prediction for sample [2 2]: 11
Prediction for sample [2 1]: 32
Prediction for sample [2 0]: 93
Prediction for sample [1 1]: 4
Prediction for sample [1 2]: 65
Prediction for sample [1 0]: 26
Prediction for sample [0 0]: 87
Prediction for sample [0 1]: 48
Prediction for sample [0 2]: 19
Prediction for sample [6 6]: 70
